In [ ]:
import tensorflow as tf
import keras
import os, shutil 
# 原始目录所在的路径
original_dataset_dir = 'C:/Users/13977/Pictures/train/'

# 数据集分类后的目录    
base_dir = '../data/cat_dog_data/' 
os.mkdir(base_dir)

# 训练、验证、测试数据集的目录
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 猫训练图片所在目录
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 猫验证图片所在目录
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 猫测试数据集所在目录
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# 狗训练图片所在目录
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# 狗验证数据集所在目录
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 狗测试数据集所在目录
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [ ]:
label = []
for img_digit in range(1, 15626):
    if os.path.exists(f"C:/Users/13977/Pictures/train/cat.{img_digit}.jpg"):
        label.append(f"cat{img_digit}")
    elif os.path.exists(f"C:/Users/13977/Pictures/train/dog.{img_digit}.jpg"):
        label.append(f"dog{img_digit}")
label

In [ ]:
import re
catlist = []
doglist = []
for i in label:
    count = re.findall(r'\d+',i)
    count = int(count[0])
    if re.match(r'cat', i):
        catlist.append(count)
    else:
        doglist.append(count)
catlist


In [ ]:
doglist

In [ ]:
fnames = []
for i in label:
    for count in catlist:
        if re.match(r"cat", i):
            if count <= 5000:
                # 将前10000张猫图像复制到train_cats_dir
                fnames.append('cat.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(train_cats_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []
                    

            if count > 5000 and count <= 6000:
                # 将下2000张猫图像复制到validation_cats_dir
                fnames.append('cat.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(validation_cats_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

            if count > 6000 and count <= 7000:
                # 将下2000张猫图像复制到test_cats_dir
                fnames.append('cat.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(test_cats_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

    for count in doglist:
        if re.match(r"dog",i):
            if count <= 5000:
                # 将前10000张狗图像复制到train_dogs_dir
                fnames.append('dog.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(train_dogs_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

            if count > 5000 and count <= 6000:
                # 将下2000张狗图像复制到validation_dogs_dir
                fnames.append('dog.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(validation_dogs_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

            if count > 6000 and count <= 7000:
                # 将下2000张狗图像复制到test_dogs_dir
                fnames.append('dog.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(test_dogs_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

In [ ]:
# 此处改成自己的路径
train_dir = '../data/cat_dog_data/train'
validation_dir = '../data/cat_dog_data/test'

In [ ]:
# 网络架构
from keras import layers, models, regularizers
from keras.callbacks import EarlyStopping

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
print(model.summary())

# 图像预处理
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

from tensorflow import optimizers
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-4),
metrics=['acc'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')
history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=15,
                              validation_data=validation_generator,
                              validation_steps=50,
                              callbacks=[early_stopping])
model.save('cats_and_dogs_small_1.h5')
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
from keras.preprocessing import image
import numpy as np
import os
from keras.models import load_model
import csv

with open('../data/submission.csv','a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'TARGET'])
        csvfile.close()
model = load_model('cats_and_dogs_small_1.h5')
test_dir = 'C:/Users/13977/Pictures/test/'
fnames = []
list = []
m = 1
for i in os.listdir(test_dir):
    fnames.append(i)
fnames.sort(key=lambda x:int(x[:-4]))
for fname in fnames:
    img_path = os.path.join(test_dir, fname)
    img = image.load_img(img_path, target_size=(150, 150))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    dogprobility = classes[0]
    with open('../data/submission.csv', 'a') as csvfile:
        writer = csv.writer(csvfile, lineterminator='\n')
        writer.writerow([m, dogprobility[0]])
        csvfile.close()
    m += 1